## Data pre-processing and data augmentation

In [1]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')

Using TensorFlow backend.


- rotation_range:在（0,180）范围内取值，限制图片旋转的范围
- width_shift and height_shift:取整个高和宽一部分的浮点数数
- rescale: 在其他任何处理之前设定的一个值，原始图片的RGB参数在0-255之间，但这个数值对于一个典型的learning rate来说还是过大，所以把原始数据乘以rescale，使范围限制在（0,1）
- shear-range:一个浮点数用于设定随意地应用剪切变换
- zoom_range:一个浮点数用于设定随意地放大图片
- horizontal_flip:True/False,用于设定是否水平翻转一半的图片
- fill_mode:‘constant’，‘nearest’，‘reflect’或‘wrap’之一，用于填充新创建的像素的策略，其可以在旋转或宽度/高度偏移之后出现

更多参数详见：http://keras-cn.readthedocs.io/en/latest/preprocessing/image/

In [7]:
from keras.preprocessing.image import ImageDataGenerator,array_to_img,img_to_array,load_img

datagen = ImageDataGenerator(
            rotation_range=40,
            width_shift_range=0.2,
            height_shift_range=0.2,
            rescale=1./255,
            shear_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            fill_mode='nearest')

img = load_img('kaggle/train/cat/cat.0.jpg') 
x = img_to_array(img) # this is a Numpy array with shape (3,150,150)
x = x.reshape((1,) + x.shape) # this is a Numpy array with shape (1,3,150,150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to be the 'preview/' directory
i = 0
for batch in datagen.flow(x,batch_size=1,save_to_dir='preview',save_prefix='cat',save_format='jpeg'):
    i += 1
    if i >20:
        break 

## Training a small convnet from scratch:in 40 lines of code

In [20]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation,Dropout,Flatten,Dense
from keras import backend as K 

# dimensions of our images.
img_width, img_height = 150, 150


train_data_dir = 'kaggle/train'
validation_data_dir = 'kaggle/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)


model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.summary()

model.compile(loss='binary_crossentropy',
             optimizer='rmsprop',
             metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
                        rescale=1./255,
                        shear_range=0.2,
                        zoom_range=0.2,
                        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in subfolers of 'kaggle/train',and indefinitely generate
# batches of augmented image data 
train_generator = train_datagen.flow_from_directory(
              train_data_dir, 
              target_size=(img_width, img_height),
              batch_size=batch_size,
              class_mode='binary') 


# this is a similar generator,for validation data
validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='binary')

model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)
model.save_weights('first_try.h5')  # always save weights after training or during training

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 148, 148, 32)      896       
_________________________________________________________________
activation_21 (Activation)   (None, 148, 148, 32)      0         
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 72, 72, 32)        9248      
_________________________________________________________________
activation_22 (Activation)   (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 34, 34, 64)        18496     
__________

## Using the bottleneck features of a pre-trained network

- 使用VGG的原因：
We will use the VGG16 architecture, pre-trained on the ImageNet dataset --a model previously featured on this blog. Because the ImageNet dataset contains several "cat" classes (persian cat, siamese cat...) and many "dog" classes among its total of 1000 classes, this model will already have learned features that are relevant to our classification problem. In fact, it is possible that merely recording the softmax predictions of the model over our data rather than the bottleneck features would be enough to solve our dogs vs. cats classification problem extremely well. However, the method we present here is more likely to generalize well to a broader range of problems, including problems featuring classes absent from ImageNet.

In [31]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras import applications

# dimensions of our images.
img_width, img_height = 150, 150


top_model_weights_path = 'bottleneck_fc_model.h5'
train_data_dir = 'kaggle/train'
validation_data_dir = 'kaggle/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50 
batch_size = 16



# save bottle features
def save_bottleneck_features():
    datagen = ImageDataGenerator(rescale=1./255)
    
    
    # bulid the VGG network
    model = applications.VGG16(include_top=False, weights = 'imagenet')
    
    generator = datagen.flow_from_directory(
                train_data_dir,
                target_size=(img_width, img_height),
                batch_size=batch_size,
                class_mode=None,
                shuffle=False)
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)
    np.save(open('bottleneck_features_train.npy', 'wb'),
                bottleneck_features_train)
    
    generator = datagen.flow_from_directory(
                validation_data_dir,
                target_size=(img_width, img_height),
                batch_size=batch_size,
                shuffle=False) # be in order 
    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples // batch_size)
    np.save(open('bottleneck_features_validation.npy', 'wb'),
                bottleneck_features_validation)
    
    
    
# train the model 
def train_top_model():
    train_data = np.load(open('bottleneck_features_train.npy', 'rb'))
    train_labels = np.array(
            [0] * int(nb_train_samples / 2) + [1] *int(nb_train_samples /2))  # set the cat label is 0, the dog labels is 1

    validation_data = np.load(open('bottleneck_features_validation.npy', 'rb'))
    validation_labels = np.array(
            [0] * int(nb_validation_samples / 2) + [1] * int(nb_validation_samples /2))


    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))


    model.compile(optimizer='rmsprop',
                        loss='binary_crossentropy', metrics=['accuracy'])

    model.fit(train_data, train_labels,
                 epochs=epochs,
                 batch_size=batch_size,
                 validation_data=(validation_data, validation_labels))
    model.save_weights(top_model_weights_path)


save_bottleneck_features()
train_top_model()
        
    

Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Train on 2000 samples, validate on 800 samples
Epoch 1/50
2000/2000 [==============================] - 13s 6ms/step - loss: 0.6613 - acc: 0.7545 - val_loss: 0.3028 - val_acc: 0.8725
Epoch 2/50
2000/2000 [==============================] - 11s 6ms/step - loss: 0.3658 - acc: 0.8535 - val_loss: 0.2511 - val_acc: 0.9012
Epoch 3/50
2000/2000 [==============================] - 11s 5ms/step - loss: 0.3097 - acc: 0.8865 - val_loss: 0.2360 - val_acc: 0.9038
Epoch 4/50
2000/2000 [==============================] - 11s 6ms/step - loss: 0.2544 - acc: 0.9050 - val_loss: 0.2791 - val_acc: 0.9087
Epoch 5/50
2000/2000 [==============================] - 11s 6ms/step - loss: 0.2533 - acc: 0.9065 - val_loss: 0.2421 - val_acc: 0.8962
Epoch 6/50
2000/2000 [==============================] - 11s 6ms/step - loss: 0.2374 - acc: 0.9120 - val_loss: 0.3021 - val_acc: 0.9100
Epoch 7/50
2000/2000 [==============================] - 11s 

## Fine-tuning the top layers of a a pre-trained network

Note that:

- in order to perform fine-tuning, all layers should start with properly trained weights: for instance you should not slap a randomly initialized fully-connected network on top of a pre-trained convolutional base. This is because the large gradient updates triggered by the randomly initialized weights would wreck the learned weights in the convolutional base. In our case this is why we first train the top-level classifier, and only then start fine-tuning convolutional weights alongside it.
- we choose to only fine-tune the last convolutional block rather than the entire network in order to prevent overfitting, since the entire network would have a very large entropic capacity and thus a strong tendency to overfit. The features learned by low-level convolutional blocks are more general, less abstract than those found higher-up, so it is sensible to keep the first few blocks fixed (more general features) and only fine-tune the last one (more specialized features).
- fine-tuning should be done with a very slow learning rate, and typically with the SGD optimizer rather than an adaptative learning rate optimizer such as RMSProp. This is to make sure that the magnitude of the updates stays very small, so as not to wreck the previously learned features.

In [ ]:
from keras.applications import VGG16
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
top_model_weights_path = 'bottleneck_fc_model.h5'
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = 'kaggle/train'
validation_data_dir = 'kaggle/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

# build the VGG16 network
model = VGG16(weights='imagenet', include_top=False,input_shape=(img_width, img_height, 3))
print('Model loaded.')
print(model.output_shape[1:])
# build a classifier model to put on top of the convolutional model
top_model = Sequential()
top_model.add(Flatten(input_shape=model.output_shape[1:]))
top_model.add(Dense(256, activation='relu'))
top_model.add(Dropout(0.5))
top_model.add(Dense(1, activation='sigmoid'))

# note that it is necessary to start with a fully-trained
# classifier, including the top classifier,
# in order to successfully do fine-tuning
top_model.load_weights(top_model_weights_path)

# add the model on top of the convolutional base
# model.add(top_model)
model = Model(inputs=model.input, outputs=top_model(model.output))

# set the first 25 layers (up to the last conv block)
# to non-trainable (weights will not be updated)
for layer in model.layers[:25]:
    layer.trainable = False

# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='binary')

# fine-tune the model
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

Model loaded.
(4, 4, 512)
Found 2000 images belonging to 2 classes.
Found 800 images belonging to 2 classes.
Epoch 1/50
125/125 [==============================] - 1201s 10s/step - loss: 0.4728 - acc: 0.9385 - val_loss: 1.0049 - val_acc: 0.8925
Epoch 2/50
125/125 [==============================] - 1214s 10s/step - loss: 0.5400 - acc: 0.9310 - val_loss: 1.0049 - val_acc: 0.8925
Epoch 3/50
125/125 [==============================] - 1195s 10s/step - loss: 0.4216 - acc: 0.9360 - val_loss: 1.0049 - val_acc: 0.8925
Epoch 4/50
125/125 [==============================] - 1188s 10s/step - loss: 0.4660 - acc: 0.9325 - val_loss: 1.0049 - val_acc: 0.8925
Epoch 5/50
125/125 [==============================] - 1177s 9s/step - loss: 0.5041 - acc: 0.9330 - val_loss: 1.0049 - val_acc: 0.8925
Epoch 6/50
125/125 [==============================] - 1132s 9s/step - loss: 0.4785 - acc: 0.9350 - val_loss: 1.0049 - val_acc: 0.8925
Epoch 7/50
125/125 [==============================] - 1126s 9s/step - loss: 0.4958 